In [3]:
import requests
import pandas as pd
import pymysql
import traceback2

In [11]:
def __connect__():
    db_user = 'coin'
    db_password = 'dragonstone#123'
    db_name = 'master_data'
    db_host = 'localhost'
    return pymysql.connect(
        unix_socket=None,
        host=db_host,
        user=db_user,
        password=db_password,
        db=db_name,
        charset='latin1',
        cursorclass=pymysql.cursors.DictCursor)

def get_cursor(connection):
    if connection is None:
        connection = __connect__()
    connection.ping(reconnect=True)
    return connection.cursor()

In [19]:
nl_df = pd.read_csv('name_lookup.csv')
source = nl_df.source.unique()
name_source_code = {}
name_source_code['Coin Market Cap'] = 'cmc'
name_source_code['Token Terminal'] = 'tt'
name_source_code['Dune Analytics'] = 'duneanalytics'
name_source_code['DeFi Pulse'] = 'defi'
name_source_code['Debank'] = 'debank'

def get_source_symbol_mapping(master_source_symbol_mapping, source, source_key):
    df = master_source_symbol_mapping[master_source_symbol_mapping['source'] == source].copy()
    df2 = pd.DataFrame([])
    df2['symbol'] = df['symbol']
    df2[f'name_in_{source_key}'] = df['source name']
    df2['name_in_cmc'] = df['name_lookup']
    df2.reset_index(inplace=True)
    del df2['index']
    return df2

source_symbols_df = {}
for source in name_source_code:
    source_symbol_df = get_source_symbol_mapping(nl_df, source, name_source_code[source])
    source_symbols_df[name_source_code[source]] = source_symbol_df

In [6]:
source_symbols_df

{'cmc':     symbol name_in_cmc
 0      MHC   #MetaHash
 1      ZCN      0Chain
 2      ZRX          0x
 3     TSHP     12Ships
 4    1INCH       1inch
 ..     ...         ...
 912   ZEON        ZEON
 913    ZEE    ZeroSwap
 914    ZIL     Zilliqa
 915    ZKS      ZKSwap
 916    ZYN    Zynecoin
 
 [917 rows x 2 columns],
 'tt':     symbol          name_in_tt         name_in_cmc
 0      ZRX                  0x                  0x
 1    1INCH               1inch               1inch
 2     AAVE                Aave                Aave
 3       AR             Arweave             Arweave
 4   BADGER           BadgerDAO              Badger
 5      BAL            Balancer            Balancer
 6      BNT              Bancor              Bancor
 7      BNB       Binance Chain        Binance Coin
 8      BTC             Bitcoin             Bitcoin
 9      ADA             Cardano             Cardano
 10     NaN          Centrifuge          Centrifuge
 11    COMP            Compound            Compo

In [15]:
def insert_master_symbol(df):
    connection = __connect__()
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print(row['symbol'], row['name_in_cmc'], row['name_in_tt'], row['name_in_debank'], row['name_in_defi'], row['name_in_duneanalytics'])
            sql = "INSERT INTO `master_symbols_2` (`symbol`, `name_in_cmc`,`name_in_tt`,`name_in_debank`,`name_in_defi`,`name_in_duneanalytics`) VALUES (%s, %s, %s, %s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_tt'], row['name_in_debank'], row['name_in_defi'], row['name_in_duneanalytics']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

In [20]:
cmc_df = source_symbols_df['cmc']
for source_symbol_df in source_symbols_df:
    if source_symbol_df != 'cmc':
#         del source_symbols_df[source_symbol_df]['name_in_cmc']
        cmc_df = pd.merge(cmc_df, source_symbols_df[source_symbol_df], on='symbol', how='left')

In [21]:
cmc_df = cmc_df.astype(object).where(pd.notnull(cmc_df), "")
cmc_df

,symbol,name_in_cmc_x,name_in_tt,name_in_cmc_y,name_in_duneanalytics,name_in_cmc_x,name_in_defi,name_in_cmc_y,name_in_debank,name_in_cmc
0,MHC,#MetaHash,,,,,,,,
1,ZCN,0Chain,,,,,,,,
2,ZRX,0x,0x,0x,,,,,,
3,TSHP,12Ships,,,,,,,,
4,1INCH,1inch,1inch,1inch,1inch,1inch,1inch-Liquidity-Protocol,1inch,1inch,1inch
...,...,...,...,...,...,...,...,...,...,...
931,ZEON,ZEON,,,,,,,,
932,ZEE,ZeroSwap,,,,,,,,
933,ZIL,Zilliqa,,,,,,,,
934,ZKS,ZKSwap,,,,,,,,


In [23]:
insert_master_symbol(cmc_df)

MHC     
ZCN     
ZRX  0x   
TSHP     
1INCH 1inch 1inch 1inch 1inch-Liquidity-Protocol 1inch
Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in rea

ELF     
AERGO     
AE     
AGA     
ADK     
AION     
AST Airswap  airswap  
AKT     
AKRO     
ACH     
ALEPH     
ALGO     
ALBT     
ALPACA     
ALPHA Alpha Homora  Alpha Homora Alpha-Homora 
AQT     
AMB     
AME     
AMO     
AMP     
AMPL     
ANW     
GOM2     
ANKR     
aEth     
MATTER     
AST Airswap  airswap  
Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9

AYA     
ASTA     
ATRI     
AWC     
ATT     
AUDIO     
REP    Augur Augur
AOA     
AUTO     
AVAX     
AXEL     
AXS     
BPC     
BADGER Badger BadgerDAO Badger DAO Badger-DAO BadgerDAO
BAKE     
BAL Balancer Balancer balancer Balancer Balancer
Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_quer

BUSD Binance USD  Binance USD  
BTRS     
BTC  Bitcoin   
BTC2     
BTCB     
BCH     
BCHA     
BCD     
BIFI     
Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.

BOR BoringDAO  BoringDAO  
Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Librar

...

GET     
Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Library/Frameworks/Pytho

TROY     
TRUE     
TRU TrueFi  TrueFi TrueFi 
Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_pack

UNI Uniswap Uniswap uniswap2 Uniswap Uniswap
UNFI     
LAYER     
UFT     
SOCKS     
UNISTAKE     
UNI Uniswap Uniswap uniswap Uniswap Uniswap
Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-p

VSYS     
VAI     
VET     
VLX     
XVS     
vBNB     
vBTC     
vBUSD     
vETH     
vLINK     
vLTC     
vSXP     
vUSDC     
vUSDT     
vXVS     
VRA     
XVG     
VERI     
VTC     
VRSC     
VSP Vesper  Vesper Vesper 
Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read(

YUSRA     
ZANO     
ZAP     
ZB     
ZEC  Zcash   
ZLW     
ZNN     
ZEON     
ZEE     
ZIL     
ZKS     
ZYN     


In [7]:
sql = """
select cmcs.`symbol` as symbol, cmcs.`name_in_cmc` as name_in_cmc, tts.`name_in_tt` as name_in_tt, debanks.`name_in_debank` as name_in_debank, defis.`name_in_defi` as name_in_defi, dunes.`name_in_duneanalytics` as name_in_duneanalytics from `cmc_symbols` cmcs
left join `tt_symbols` tts on tts.`symbol` = cmcs.`symbol`
left join `debank_symbols` debanks on debanks.`symbol` = cmcs.`symbol`
left join `defi_symbols` defis on defis.`symbol` = cmcs.`symbol`
left join `dune_symbols` dunes on dunes.`symbol` = cmcs.`symbol`
"""

In [12]:
joined_df = pd.read_sql_query(sql, __connect__())

In [17]:
joined_df = joined_df.astype(object).where(pd.notnull(joined_df), "")

In [18]:
insert_master_symbol(joined_df)

$ANRX AnRKey X    
$BASED Based Money    
$COIN COIN    
$KING King Swap    
$NOOB noob.finance    
$PIR PIRANHAS    
$ROPE Rope    
$TRDL Strudel Finance    
0xBTC 0xBitcoin    
1AI 1AI Token    
1GOLD 1irstGold    
1INCH 1inch 1inch 1inch 1inch-Liquidity-Protocol 1inch
1INCH 1inch 1inch 1INCH Token 1inch-Liquidity-Protocol 1inch
1INCH 1inch 1inch 1inch V1 1inch-Liquidity-Protocol 1inch
1MT 1Million Token    
1SG 1SG    
1ST FirstBlood    
1UP Uptrennd    
1WO 1World    
1X2 1X2 COIN    
2GIVE 2GIVE    
2KEY 2key.network    
3Cs Crypto Cricket Club    
3XT 3XT TOKEN    
42 42-coin    
4ART 4ART Coin    
777 Jackpot    
7ADD HoldToWin    
7E 7Eleven    
7UP 7up Finance    
808TA 808TA    
888 888tron    
A Alpha Token    
A2A A2A    
A5T Alpha5    
AAA Abulaba    
AAB AAX Token    
AAC Acute Angle Cloud    
AAPL Apple tokenized stock Bittrex    
AAPL Apple tokenized stock FTX    
AAVE Aave Aave Aave Token Aave Aave
AAVE Aave Aave Aave V1 Aave Aave
AAVE Aave Aave Aave V2 Aave Aave
AAVED

AT ABCC Token    
AT Artfinity    
ATB ATBCoin    
ATC Astosch    
ATCC ATC Coin    
ATH Atheios    
ATIS Atlantis Token    
ATL ATLANT    
ATLS Atlas Token    
ATM Atletico De Madrid Fan Token    
ATMOS Atmos    
ATN ATN    
ATOM Cosmos    
ATOMBEAR 3X Short Cosmos Token    
ATOMBULL 3X Long Cosmos Token    
ATP Alaya    
ATP Atlas Protocol    
ATRI Atari Token    
ATT Attila    
ATTN ATTN    
ATUSD Aave TUSD    
AUC Auctus   Auctus 
AUCTION Bounce Token    
AUDIO Audius    
AUDT Australian Dollar Token    
AUNIT Aunite    
AUR Auroracoin    
AUSCM Auric Network    
AUTO Auto    
AUTO Cube    
AUX Auxilium    
AV AvatarCoin    
AVA Travala.com    
AVAX Avalanche    
AVC AVCCOIN    
AVT Aventus    
AWC Atomic Wallet Coin    
AWG AurusGOLD    
aWSB aWSB    
AWX AurusDeFi    
AXE Axe    
AXEL AXEL    
AXI Axioms    
AXIA Axia Protocol    
AXIOM Axiom    
AXIS Axis DeFi    
AXL Axial Entertainment Digital Asset    
AXN Axion    
AXPR AXPR    
AXS Axie Infinity    
AYA Aryacoin    
AZ Azbi

BLACK eosBLACK    
BLANK Blank Wallet    
BLAST BLAST    
BLAZR BlazerCoin    
BLC Blakecoin    
BLCT Bloomzed Loyalty Club Ticket    
BLFI BlackFisk    
BLINK BLink    
BLINK Blockmason Link    
BLK BlackCoin    
BLOC BLOC.MONEY    
BLOC Blockcloud    
BLOCK Blocknet    
BLOOD Blood    
BLOODY Bloody Token    
BLTG Block-Logic    
BLU BlueCoin    
BLUE Blue Protocol    
BLUR Blur    
BLURT Blurt    
BLV Bellevue Network    
BLVR BELIEVER    
BLX Balloon-X    
BLY Blocery    
BLZ Bluzelle    
BLZN Blaze Network    
BMC Blackmoon    
BMH BlockMesh    
BMI Bridge Mutual    
BMP Brother Music Platform    
BMX BitMart Token    
BMXX Multiplier    
BNA Bananatok    
BNANA Chimpion    
BNB Binance Coin Binance Chain   
BNBBEAR 3X Short BNB Token    
BNBBULL 3X Long BNB Token    
BNBDOWN BNBDOWN    
BNBUP BNBUP    
BNC Bionic    
BNF BonFi    
BNFI Blaze DeFi    
BNK Bankera    
BNKR Bankroll Network    
BNKY Bonkey dAPP    
BNOX BlockNoteX    
BNP BenePit Protocol    
BNS BNS Token    
BNSD 

BZKY BIZKEY    
BZNT Bezant    
BZRX bZx Protocol  bZx Protocol Token bZx 
BZX Bitcoin Zero    
C2 Coin2.1    
C20 CRYPTO20    
C2O CryptoWater    
CAB Cabbage    
CAD Candy Protocol    
CADC CAD Coin    
CAI Club Atletico Independiente    
CAJ Cajutel    
CAKE PancakeSwap  PancakeSwap  
CALL Global Crypto Alliance    
CAMO Camouflage.eth    
CAMP Camp    
CAN CanYaCoin    
CAN Channels    
CAN Coinwaycoin    
CANN CannabisCoin    
CAP Cap    
CAP Capital.Finance    
CAPP Cappasity    
CARAT CARAT    
CARBON Carboncoin    
CARD Cardstack    
CARE Carebit    
CAS Cashaa    
CASH Litecash    
CAT Cat Token    
CATT Catex Token    
CATX CAT.trade Protocol    
CAVO Excavo Finance    
CBANK CryptoBank    
CBAT Compound Basic Attention Token    
CBC Cashbery Coin    
CBC Casino Betting Coin    
CBC CryptoBharatCoin    
CBC CryptoBossCoin    
CBET CryptoBet    
CBIX Cubiex    
CBK Cobak Token    
CBM CryptoBonusMiles    
CBP CashBackPro    
CBR Cybercoin    
CBSE Coinbase Pre-IPO tokenized st

Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Library/Frameworks/Python.framewo

...

Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Library/Frameworks/Python.framewo

Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Library/Frameworks/Python.framewo

Traceback (most recent call last):
  File "<ipython-input-15-8837588dc7e8>", line 8, in insert_master_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Library/Frameworks/Python.framewo

XGM Defis    
XGS GenesisX    
XGT Xion Finance    
XHI HiCoin    
XHV Haven Protocol    
XIF X Infinity    
XIN Infinity Economics    
XIN Mixin    
XIND INDINODE    
XIO Blockzero Labs    
XIOT Xiotri    
XIV Project Inverse    
XLA Ripple Alpha    
XLA Scala    
XLAB XcelToken Plus    
XLM Stellar Stellar   
XLMDOWN XLMDOWN    
XLMG Stellar Gold    
XLMUP XLMUP    
XLPG StellarPayGlobal    
XLQ ALQO    
XLR Solaris    
XLT Nexalt    
XMC Monero Classic    
XMCT XMCT    
XMM Momentum    
XMON XMON    
XMR Monero    
XMV MoneroV     
XMX XMax    
XMY Myriad    
XNB Xeonbit    
XNC XeniosCoin    
XNK Ink Protocol    
XNO Xeno Token    
XNODE XNODE    
XNS Xeonbit Token    
XNV Nerva    
XOR Oracolxor    
XOR Sora    
XOV XOVBank    
XPA XPA    
XPAT Bitnation    
XPB Transmute Protocol    
XPC eXPerience Chain    
XPD PetroDollar    
XPM Primecoin    
XPN PANTHEON X    
XPR Proton    
XPT Cryptobuyer    
XPT XPToken.io    
XPTX PlatinumBAR    
XPX ProximaX    
XPY PayCoin    
XQC QURAS

YFIM Yfi.mobi    
YFIS YFISCURITY    
YFIUP YFIUP    
YFIVE YFIVE FINANCE    
YFIX YFIX Finance    
YFKA Yield Farming Known as Ash    
YFL YF Link  YFLink  
YFMB YFMoonBeam    
YFMS YFMoonshot    
YFN Yearn Finance Network    
YFO YFIONE    
YFOS YFOS.finance    
YFOX YFOX FINANCE    
YFP Yearn Finance Protocol    
YFPI Yearn Finance Passive Income    
YFPRO YFPRO Finance    
YFRM Yearn Finance Red Moon    
YFSI Yfscience    
YFST YFST.Protocol    
YFT Toshify.finance    
YFT Yield Farming Token    
YFTE YFTether    
YFUEL YFUEL    
YFW YFWorld    
YI12 Yield Stake Finance    
YLAND Yearn Land    
YLC YoloCash    
YLD Yield    
YLD YIELD App    
YLFI Yearn Loans Finance    
YMAX YMAX    
YMEN Ymen.Finance    
YMPL YMPL    
YNI YEARNYFI NETWORK    
YNK Yoink    
YO Yobit Token    
YOC Yocoin    
YOP Yield Optimization Platform & Protocol    
YOT PayYoda    
YOU YOU COIN    
YOUC yOUcash    
YOYOW YOYOW    
yPANDA YieldPanda Finance    
YPIE PieDAO Yearn Ecosystem Pie    
YPLT yplutus  

In [ ]:
## Below are the manual symbols projects mapping.

In [3]:
token_terminal_symbol_name = []
url='https://api-public.tokenterminal.com/latest_metrics'
res = requests.get(url)
json_objs = res.json()
for obj in json_objs:
    token_terminal_symbol_name.append([obj['name'],obj['symbol']])


In [4]:
tt_symbol_name_df = pd.DataFrame(token_terminal_symbol_name, columns=['name','symbol'])

In [5]:
tt_symbol_name_df

,name,symbol
0,0x,ZRX
1,1inch,1INCH
2,Aave,AAVE
3,Balancer,BAL
4,Bancor,BNT
5,Binance Chain,BNB
6,Bitcoin,BTC
7,Centrifuge,RAD
8,Compound,COMP
9,Cream,CREAM


In [6]:
db_project_symbol_name = []
url='https://api.debank.com/project/v2/list'
res=requests.get(url)
data = res.json()['data']
for d in data:
    db_project_symbol_name.append([d['id'], d['name']['en']])

In [7]:
db_project_symbol_name_df = pd.DataFrame(db_project_symbol_name, columns=['debank_proj_id', 'name'])
db_project_symbol_name_df

,debank_proj_id,name
0,0x,0x
1,1inch,1inch V1
2,1inch2,1inch
3,1sd,1sd
4,88mph,88mph V1
...,...,...
283,zai,ZAI V1
284,zai2,ZAI V2
285,zelda,Zelda
286,zeroset,Zero Set Dollar


In [8]:
db_coin_symbol_name = []
url='https://api.debank.com/market/token_list_v2'
res=requests.get(url)
data = res.json()['data']
for d in data:
    db_coin_symbol_name.append([d['symbol'], d['name']])

In [9]:
db_coin_symbol_name_df = pd.DataFrame(db_coin_symbol_name, columns=['symbol', 'name'])
db_coin_symbol_name_df

,symbol,name
0,UNI,Uniswap
1,LINK,ChainLink Token
2,RSR,Reserve Rights
3,AAVE,Aave Token
4,1INCH,1INCH Token
...,...,...
155,BSGS,BSGS
156,ARTT,Artem
157,ZAI,Zero Collateral Dai
158,SAS,SAS


In [10]:
api_key = "a86e3326-8c8d-434e-897a-5d6becbf1f35"
def fetch_top_n_market_cap_symbols_(n):
    url = 'https://pro-api.coinmarketcap.com/v1/cryptocurrency/listings/latest'
    payload = {
        'convert':'USD',
        'aux':'cmc_rank',
        'limit':n
    }
    headers = {
      'Accepts': 'application/json',
      'X-CMC_PRO_API_KEY': api_key
    }
    
    data = requests.get(url, params=payload, headers=headers).json()['data']
    return data

data = fetch_top_n_market_cap_symbols_(5000)
cmc_symbol_name = []
for d in data:
    cmc_symbol_name.append([d['symbol'], d['name']])
    
cmc_symbol_name_df = pd.DataFrame(cmc_symbol_name, columns=['symbol', 'name'])
cmc_symbol_name_df

,symbol,name
0,BTC,Bitcoin
1,ETH,Ethereum
2,BNB,Binance Coin
3,USDT,Tether
4,ADA,Cardano
...,...,...
4331,wSZO,ShuttleOne
4332,aENJ,Aave Enjin
4333,WEATHER,Weather Finance
4334,WANSUSHI,wanSUSHI


#### Insert Coinmarketcap Symbol Table data.

In [11]:
def insert_cmc_symbol(df):
    connection = __connect__('coinmarketcap')
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print(row['symbol'], row['name'])
            sql = "INSERT INTO `cmc_symbols` (`symbol`, `name_in_cmc`) VALUES (%s, %s)"
            cursor.execute(sql, (row['symbol'], row['name']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

In [12]:
insert_cmc_symbol(cmc_symbol_name_df)

BTC Bitcoin
ETH Ethereum
BNB Binance Coin
USDT Tether
ADA Cardano
DOT Polkadot
XRP XRP
UNI Uniswap
LTC Litecoin
LINK Chainlink
BCH Bitcoin Cash
USDC USD Coin
XLM Stellar
WBTC Wrapped Bitcoin
LUNA Terra
DOGE Dogecoin
THETA THETA
VET VeChain
AAVE Aave
CRO Crypto.com Coin
ATOM Cosmos
XMR Monero
EOS EOS
FTT FTX Token
AVAX Avalanche
MIOTA IOTA
TRX TRON
SOL Solana
BSV Bitcoin SV
XEM NEM
CHZ Chiliz
FIL Filecoin
BUSD Binance USD
ALGO Algorand
XTZ Tezos
DAI Dai
HBAR Hedera Hashgraph
NEO Neo
KSM Kusama
HT Huobi Token
SUSHI SushiSwap
ENJ Enjin Coin
EGLD Elrond
DASH Dash
SNX Synthetix
DCR Decred
GRT The Graph
MKR Maker
LEO UNUS SED LEO
COMP Compound
NEAR NEAR Protocol
TFUEL Theta Fuel
MATIC Polygon
RVN Ravencoin
ZIL Zilliqa
BTT BitTorrent
MANA Decentraland
ZEC Zcash
NEXO Nexo
ETC Ethereum Classic
BTCB Bitcoin BEP2
RUNE THORChain
UMA UMA
CAKE PancakeSwap
YFI yearn.finance
CHSB SwissBorg
BNT Bancor
VGX Voyager Token
STX Stacks
HOT Holo
ICX ICON
BAT Basic Attention Token
CEL Celsius
UST TerraUSD
REN 

BNK Bankera
ORAI Oraichain Token
NWC Newscrypto
EFX Effect.AI
VTC Vertcoin
BUX BUX Token
PRO Propy
PAI Project Pai
NAV Navcoin
RDN Raiden Network Token
BIP Minter Network
AOA Aurora
BAO Bao Finance
VERI Veritaseum
MOF Molecular Future
NSBT Neutrino Token
ZEE ZeroSwap
FXF Finxflo
PRE Presearch
COVER COVER Protocol
RFR Refereum
AMO AMO Coin
TXL Tixl
BOT Bounce Token (OLD)
RBTC RSK Smart Bitcoin
CSC CasinoCoin
TOP TOP
PEAK PEAKDEFI
LGO LGO Token
GVT Genesis Vision
ULT Ultiledger
RARI Rarible
TONE TE-FOOD
MET Metronome
BTC2 Bitcoin 2
vUSDT Venus USDT
STAX StableXSwap
RBC Rubic
OPIUM Opium
BFT BnkToTheFuture
NFT NFT
GTO Gifto
BASIC BASIC
DVP Decentralized Vulnerability Platform
PMA PumaPay
AGA AGA Token
BLOCK Blocknet
DYN Dynamic
ALEPH Aleph.im
LGCY LGCY Network
PSG Paris Saint-Germain Fan Token
IHF Invictus Hyperion Fund
USDK USDK
OAX OAX
FOR ForTube
TCT TokenClub
MITX Morpheus Labs
SLT Smartlands Network
ANW Anchor Neural World
WPR WePower
QASH QASH
FWT Freeway Token
MATTER AntiMatter
AST

XPN PANTHEON X
OCE OceanEx Token
CRW Crown
AAC Acute Angle Cloud
BITCNY bitCNY
TERC TronEuropeRewardCoin
FEX FidexToken
DGTX Digitex
DGX Digix Gold Token
IDEA Ideaology
ERC20 ERC20
YETH fyeth.finance
CBIX Cubiex
PCNT Playcent
BWT Bittwatt
ALPACA Alpaca Finance
CATT Catex Token
BOX ContentBox
UGAS UGAS
CRBN Carbon
BLK BlackCoin
SLP Small Love Potion
GHOST GHOST
MCX Machi X
CHADS CHADS VC
PIZZA Pizza
MASQ MASQ
CAJ Cajutel
BITG BitGreen
LOOT NFTLootBox
ESS Essentia
KCAL Phantasma Energy
EPIC Epic Cash
OK OKCash
ASTRO AstroTools
IDRT Rupiah Token
AIDUS AIDUS TOKEN
PPY Peerplays
HLC HalalChain
REPO REPO
RVC Ravencoin Classic
HYN Hyperion
AMN Amon
CAI Club Atletico Independiente
INXT Internxt
DAX DAEX
DAT Datum
QUN QunQun
BPC BackPacker Coin
COIN Coin Artist
KEBAB Kebab Token
PMEER Qitmeer
DMG DMM: Governance
PINK Pinkcoin
THUGS Thugs Finance
GAP GAPS
YEE YEE
CHX WeOwn
ADB adbank
BIFI BiFi
SHARE Seigniorage Shares
NOKU Noku
GMB GAMB
HOMI HOMIHELP
VI Vid
MOONS MoonTools
AXPR AXPR
BLINK BLink


SHDW Shadow Token
OPT Opus
SCL Sociall
ISLA Insula
TON TON Token
KICKS Sessia
CRP Crypton
HGT HelloGold
MERI Merebel
EDG Edgeless
NFXC NFX Coin
GLS Golos Blockchain
ZENI Zennies
X8X X8X Token
ZSC Zeusshield
ADC AudioCoin
BOB Bob's Repair
MCT Master Contract Token
POSS Posscoin
PTON PTON
UP UpToken
MAX MAX Exchange Token
ELET Elementeum
TCORE Tornado
BLTG Block-Logic
BTDX Bitcloud
BDK Bidesk
EVE Devery
XNV Nerva
CHESS ChessCoin
ACM Actinium
IMT Moneytoken
BIRD Birdchain
DAV DAV Coin
CRAD CryptoAds Marketplace
TIX Blocktix
NKC Nework
IETH iEthereum
XWP Swap
PIPL PiplCoin
CCN CustomContractNetwork
ETHM Ethereum Meta
WCK Wrapped Basic CryptoKitties
GPYX GoldenPyrex
ROAD ROAD
MNTP GoldMint
CSPN Crypto Sports
RATING DPRating
XDNA extraDNA
FGC FantasyGold
VGW VegaWallet Token
ALT Alt.Estate token
BDG BitDegree
KFX KnoxFS (new)
SPHTX SophiaTX
UPX uPlexa
AMA AMATEN
KP4R Keep4r
CVCC CryptoVerificationCoin
PUB PUBLYTO Token
SDS Alchemint Standards
LTHN Lethean
PENG PENG
SPD Stipend
RIGEL Rigel Fi

CSTL Castle
TAJ TajCoin
SQR Squorum
STREAM Streamit Coin
$NOOB noob.finance
QUAN Quantis Network
DTC Datacoin
KKC Kabberry Coin
XBTC21 Bitcoin 21
NAVY BoatPilot Token
BURN Blockburn
PNX Phantomx
onLEXpa onLEXpa
ZYD Zayedcoin
EUC Eurocoin
HSN Hyper Speed Network
IMPL Impleum
PAZZI Paparazzi
EXO Exosis
XPTX PlatinumBAR
CREVA CrevaCoin
XDNA XDNA
KUN KUN
PLNC PLNcoin
PAYX Paypex
EGX EagleX
SPRKL Sparkle Loyalty
HELP GoHelpFund
FTXT FUTURAX
SONG SongCoin
FLC Flowchain
NYEX Nyerium
SHB SkyHub Coin
KWATT 4NEW
XOV XOVBank
ELD Electrum Dark
BITS Bitcoinus
FLL Feellike
ORE Galactrum
1X2 1X2 COIN
aWSB aWSB
LTCR Litecred
UVU CCUniverse
TVNT TravelNote
GRIMM Grimm
KTS Klimatas
MRI Mirai
BCAC Business Credit Alliance Chain
QBC Quebecoin
SCS SpeedCash
IRA Diligence
XGS GenesisX
LST LuckySevenToken
BUL Bulleon
PXI Prime-XI
MILO MiloCoin
IBS IBStoken
CAB Cabbage
YLC YoloCash
DSR Desire
CCO Ccore
ETHP ETHPlus
VLT Veltor
JS JavaScript Token
XMCT XMCT
LUNA Luna Coin
SOVE Soverain
VEC2 VectorAI
HVCO High V

NIF Unifty
CYF CY Finance
HFI HecoFi
CHAIN Chain Games
REL Relevant
BNBBULL 3X Long BNB Token
ONX OnX Finance
FF1 Two Prime FF1 Token
TOM TOM Finance
BOG Bogged Finance
arNXM Armor NXM
CTC Culture Ticket Chain
LRG Largo Coin
ONES OneSwap DAO Token
KAL Kaleido
MXW Maxonrow
LBK LBK
MT MyToken
MATICBULL 3X Long Matic Token
CBANK CryptoBank
RAMEN RamenSwap
N0001 nHBTC
CNFI Connect Financial
TRYB BiLira
BQT Blockchain Quotations Index Token
FAR Farmland Protocol
MGP MangoChain
ARTH ARTH
PIE DeFiPie
VOICE Voice Token
FLASH Flashstake
BSE Buy-Sell
SUP SUP
KEYT REBIT
ATOMBULL 3X Long Cosmos Token
OCT Octree
ELX Energy Ledger
OBR Order of the Black Rose
TAP Tapmydata
MVC MileVerse
RUPEE HyruleSwap
EGCC Engine
BANANA Banana.finance
SHE ShineChain
BANK Bankcoin
BLOC Blockcloud
BERRY Berry
COR Coreto
CRX CryptEx
BNS BNS Token
DEA DEUS Finance DEA
BDCC BDCC Bitica COIN
MINTY Minty Art
EOSBEAR 3x Short EOS Token
TSLA Tesla tokenized stock FTX
PHO Phoswap
AKITA Akita Inu
CCAKE CheesecakeSwap Token
SP

BCNT Bincentive
FIT FIRST INTERCHANGEABLE TOKEN
TUBE BitTube
DSWAP Definex
HYDRO Hydro
WLEO Wrapped LEO
LNT Lottonation
F1C Future1coin
NAN NanTrade
GBX Gbrick
CATX CAT.trade Protocol
JDB Justdobet
BBC BigBang Core
DRGB Dragonbit
BCEO bitCEO
IX X-Block
DOOS DOOS TOKEN
GM GM Holding
MIDAS Midas
PWC Prime Whiterock Company
FME FME
ALTBULL 3X Long Altcoin Index Token
LYFE LYFE
VBNT Bancor Governance Token
OTB OTCBTC Token
GVY Groovy Finance
PTM Potentiam
UAT UltrAlpha
KONJ KONJUNGATE
METRIC Metric Exchange
MGC MGC Token
POKE Pokeball
IZE IZE
IOEX IOEX
MBC Marblecoin
ETCBULL 3X Long Ethereum Classic Token
LUXO LUXOCHAIN
XDAG XDAG
SPORE Spore Engineering
YRISE yRise Finance
STP STPAY
ETHBN EtherBone
MOONDAY Moonday Finance
FR Freedom Reserve
TUNE TUNE TOKEN
HP Healing Plus
VCCO Vera Cruz Coin
GVE Globalvillage Ecosystem
JOYS Joys Digital
BYTS Bytus
BVL Bullswap Exchange
VKF VKF Platform
SERGS SERGS
YFOX YFOX FINANCE
DMD Diamond
NIA Nydronia
1GOLD 1irstGold
GUAP Guapcoin
DFIO DeFi Omega
CVT 

CREX Crex Token
NOVO Novo
RIFT RIFT Token
RFCTR Reflector.Finance
IDL IDL Token
DSS Defi Shopping Stake
GC Gric Coin
TAXI Taxi
TTN TheToken.Network
BCNA BitCanna
iOWN iOWN Token
WIKI Wiki Token
ESWA EasySwap
DEQ Dequant
XFYI XCredit
ACE ACE
SSL Sergey Save Link
OPM Omega Protocol Money
CYTR Cyclops Treasure
GOOGL Google tokenized stock FTX
YMEN Ymen.Finance
DDR DigiDinar
YPLT yplutus
RAUX ErcauX
SPY SPDR S&P 500 ETF tokenized stock Bittrex
MACH MACH Project
ITEN ITEN
ABNB Airbnb tokenized stock FTX
GOOGL Google tokenized stock Bittrex
ZCRT ZCore Token
NFLX Netflix tokenized stock FTX
STONK STONK
PUT Profile Utility Token
WPX Wallet Plus X
FLASH Flash
TRP Tronipay
IOV IOV BlockChain
TCH THECASH
SKC SKINCHAIN
DCL Delphi Chain Link
SATX SatoExchange Token
ESK Eska
NODE PocketNode
ORS ORS Group
SENSE Sense
ZIPT Zippie
XSP XSwap
WSHIFT Wrapped Shift
XTRM XTRM COIN
NOKN Nokencoin
EDS Endorsit
EXT Experience Token
ZAI Zero Collateral Dai
ZZZV2 zzz.finance v2
CCE CloudCoin
GZE GazeCoin
BELT Be

NEAL Coineal Token
GIG Krios
TRAT Tratin
B91 B91
XSPC SpectreSecurityCoin
USDQ USDQ
SPIN SPIN Protocol
MAPR Maya Preferred 223
EVC Eco Value Coin
YTA YottaChain
GEX GLOBEX
VOL Volume Network
XENO Xenoverse
CUST Custody Token
CTT CITEX Token
ACU Aitheon
RITO Ritocoin
BSOV BitcoinSoV
BTCX Bitscoin
AER Aeryus
KBOT Korbot
ETNX Electronero
VENA VENA
LN LINK
TCH Tchain
STK STK Coin
SIERRA Sierracoin
CBC CryptoBossCoin
TKX Tokenize Xchange
VERS Versess Coin
ILK INLOCK
QPY QPay
BNY Bancacy
NOVA NOVA
DSC Dash Cash
BXT BITFXT COIN
MLGC Marshal Lion Group Coin
SBT SOLBIT
BITN Bitcoin & Company Network
BNKR Bankroll Network
XBG Bitgrin
MZG Moozicore
NIRX NairaX
SEED Sesameseed
TDPS Tradeplus
ANK Apple Network
BCB Building Cities Beyond Blockchain
CBUCKS CRYPTOBUCKS
VMR VOMER
1AI 1AI Token
FRED FRED Energy
AfroX AfroDex
KYF Kryptofranc
BFF Bitcoffeen
XT ExtStock Token
KTC Kratscoin
CSAI Compound SAI
ASAC Asac Coin
imBTC The Tokenized Bitcoin
SSN SuperSkynet
EER Ethereum eRush
ODE ODE
SAPP SappChain

PSHP Payship
BNZ BonezYard
BTCDOM Bitfinex Bitcoin Dominance Perps
PUSD PayUSD
PAYOU Payou Finance
MCN McNetworkDefi
FWB Friends With Benefits
yBAN BananoDOS
NAZ Naz Coin
BSWAP BSCswap
LMS Lumos
PXUSD_MAR2021 pxUSD Synthetic USD Expiring 1 April 2021
TAT Tatcoin
HD HubDao
BLOODY Bloody Token
HOO Hoo Token
GBCR Gold BCR
xDOT DotBased
YEET $YEET
stFIRO stakedFIRO
SMPL Smpl foundation
ZLOT zLOT Finance
FLA Fiola
YFB2 Yearn Finance Bit2
WHOLE wormhole.finance
FCD Future-Cash Digital
XFII XFII
SVN 7Finance
BST1 Blueshare Token
VOLTS Volts.Finance
KNV Kanva
PBS PBS Chain
LCT Light Coin Exchange Token
UNOS UnoSwap
LK3R Lock3r Network
WZEC Wrapped ZEC
7ADD HoldToWin
TBP Tradebitpay
LELE LeLeFoodChain
BPT Bitpumps Token
STV STVKE
PIS PiSwap Token
MTTCOIN Money of Tomorrow, Today
PSK PoolStake
PUNT Greenheart Punt
DGP DGPayment
WAV3 WAV3
ELYX Elynet
SUGAR Sugarchain
YETI Yearn Ecosystem Token Index
ARVO ARVO
Z502 502 Bad Gateway Token
vXRP Venus XRP
BNFI Blaze DeFi
TRI Trinity Protocol
MEDI Medi

In [13]:
tt_in_cmc_symbol_name_df = cmc_symbol_name_df[cmc_symbol_name_df['symbol'].isin(tt_symbol_name_df['symbol'])].copy()

In [14]:
len(token_terminal_symbol_name)

50

In [15]:
tt_in_cmc_symbol_name_df.reset_index(inplace=True)
del tt_in_cmc_symbol_name_df['index']

In [16]:
def insert_tt_cmc_symbol(df):
    connection = __connect__('tokenterminal')
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_tt']])
            sql = "INSERT INTO `tt_symbols` (`symbol`, `name_in_cmc`, `name_in_tt`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_tt']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

In [17]:
tt_cmc_coin_symbol_df = pd.merge(tt_in_cmc_symbol_name_df, tt_symbol_name_df, on='symbol', how='outer', suffixes=('_in_cmc', '_in_tt'), indicator=True)
tt_cmc_coin_symbol_df = tt_cmc_coin_symbol_df.astype(object).where(pd.notnull(tt_cmc_coin_symbol_df), "")
insert_tt_cmc_symbol(tt_cmc_coin_symbol_df)

['BTC', 'Bitcoin', 'Bitcoin']
['ETH', 'Ethereum', 'Ethereum']
['BNB', 'Binance Coin', 'Binance Chain']
['DOT', 'Polkadot', 'Polkadot']
['UNI', 'Uniswap', 'Uniswap']
['UNI', 'Universe', 'Uniswap']
['UNI', 'UNICORN Token', 'Uniswap']
['LTC', 'Litecoin', 'Litecoin']
['XLM', 'Stellar', 'Stellar']
['LUNA', 'Terra', 'Terra']
['LUNA', 'Luna Coin', 'Terra']
['DOGE', 'Dogecoin', 'Dogecoin']
['AAVE', 'Aave', 'Aave']
['FIL', 'Filecoin', 'Filecoin']
['XTZ', 'Tezos', 'Tezos']
['KSM', 'Kusama', 'Kusama']
['SUSHI', 'SushiSwap', 'SushiSwap']
['SNX', 'Synthetix', 'Synthetix']
['DCR', 'Decred', 'Decred']
['MKR', 'Maker', 'MakerDAO']
['COMP', 'Compound', 'Compound']
['COMP', 'Compound Coin', 'Compound']
['ZEC', 'Zcash', 'Zcash']
['UMA', 'UMA', 'UMA']
['YFI', 'yearn.finance', 'yearn.finance']
['BNT', 'Bancor', 'Bancor']
['REN', 'Ren', 'Ren']
['ZRX', '0x', '0x']
['LRC', 'Loopring', 'Loopring']
['1INCH', '1inch', '1inch']
['CRV', 'Curve DAO Token', 'Curve']
['KNC', 'Kyber Network', 'Kyber']
['DODO', 'DODO',

In [18]:
dbc_in_cmc_sn_df = cmc_symbol_name_df[cmc_symbol_name_df['symbol'].isin(db_coin_symbol_name_df['symbol'])].copy()
dbc_in_cmc_sn_df.reset_index(inplace=True)
del dbc_in_cmc_sn_df['index']

In [19]:
def insert_debank_coin_cmc_symbol(df):
    connection = __connect__('debank')
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_debank']])
            sql = "INSERT INTO `debank_symbols` (`symbol`, `name_in_cmc`, `name_in_debank`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_debank']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

In [20]:
dbc_cmc_coin_symbol_df = pd.merge(dbc_in_cmc_sn_df, db_coin_symbol_name_df, on='symbol', how='outer', suffixes=('_in_cmc', '_in_debank'), indicator=True)
dbc_cmc_coin_symbol_df = dbc_cmc_coin_symbol_df.astype(object).where(pd.notnull(dbc_cmc_coin_symbol_df), "")
insert_debank_coin_cmc_symbol(dbc_cmc_coin_symbol_df)

['UNI', 'Uniswap', 'Uniswap']
['UNI', 'Universe', 'Uniswap']
['UNI', 'UNICORN Token', 'Uniswap']
['LINK', 'Chainlink', 'ChainLink Token']
['AAVE', 'Aave', 'Aave Token']
['SUSHI', 'SushiSwap', 'SushiToken']
['SNX', 'Synthetix', 'Synthetix Network Token']
['MKR', 'Maker', 'Maker']
['COMP', 'Compound', 'Compound']
['COMP', 'Compound Coin', 'Compound']
['UMA', 'UMA', 'UMA Voting Token v1']
['YFI', 'yearn.finance', 'yearn.finance']
['BNT', 'Bancor', 'Bancor Network Token']
['HOT', 'Holo', 'Hydro Protocol Token']
['HOT', 'Hydro Protocol', 'Hydro Protocol Token']
['REN', 'Ren', 'Republic Token']
['RSR', 'Reserve Rights', 'Reserve Rights']
['ZRX', '0x', '0x Protocol Token']
['LRC', 'Loopring', 'LoopringCoin V2']
['1INCH', '1inch', '1INCH Token']
['CRV', 'Curve DAO Token', 'Curve DAO Token']
['ZKS', 'ZKSwap', 'Zks']
['KNC', 'Kyber Network', 'Kyber Network Crystal']
['ALPHA', 'Alpha Finance Lab', 'AlphaToken']
['DODO', 'DODO', 'DODO bird']
['BAL', 'Balancer', 'Balancer']
['BADGER', 'Badger DAO',

In [21]:
df1 = cmc_symbol_name_df.copy()
df1['name_in_cmc'] = df1['name']
df2 = db_project_symbol_name_df.copy()
df2['name_in_debank'] = db_project_symbol_name_df['name']

In [22]:
dbp_in_cmc_sn_df = df1[df1['name'].isin(df2['name'])].copy()
dbp_in_cmc_sn_df.reset_index(inplace=True)
del dbp_in_cmc_sn_df['index']

In [23]:
dbp_in_cmc_sn_df

,symbol,name,name_in_cmc
0,UNI,Uniswap,Uniswap
1,SUSHI,SushiSwap,SushiSwap
2,SNX,Synthetix,Synthetix
3,MKR,Maker,Maker
4,COMP,Compound,Compound
...,...,...,...
88,BSD,Basis Dollar,Basis Dollar
89,ONC,One Cash,One Cash
90,RSV,Reserve,Reserve
91,UNC,Unicrypt,Unicrypt


In [24]:
dbp_cmc_sybols_df = pd.merge(dbp_in_cmc_sn_df, df2, on='name', how='outer', suffixes=('_in_cmc', '_in_debank'), indicator=True)

dbp_cmc_sybols_df = dbp_cmc_sybols_df.astype(object).where(pd.notnull(dbp_cmc_sybols_df), "")
insert_debank_coin_cmc_symbol(dbp_cmc_sybols_df)


['UNI', 'Uniswap', 'Uniswap']
Traceback (most recent call last):
  File "<ipython-input-19-c7d64fffcc6e>", line 8, in insert_debank_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  F

['REP', 'Augur', 'Augur']
['ANT', 'Aragon', 'Aragon']
['AMPL', 'Ampleforth', 'Ampleforth']
['GNO', 'Gnosis', 'Gnosis']
['SWAP', 'TrustSwap', 'TrustSwap']
['ORN', 'Orion Protocol', 'Orion Protocol']
Traceback (most recent call last):
  File "<ipython-input-19-c7d64fffcc6e>", line 8, in insert_debank_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Librar

['DOS', 'DOS Network', 'DOS Network']
['BAC', 'Basis Cash', 'Basis Cash']
['IDLE', 'Idle', 'Idle']
Traceback (most recent call last):
  File "<ipython-input-19-c7d64fffcc6e>", line 8, in insert_debank_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", lin

Traceback (most recent call last):
  File "<ipython-input-19-c7d64fffcc6e>", line 8, in insert_debank_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Library/Frameworks/Pytho

Traceback (most recent call last):
  File "<ipython-input-19-c7d64fffcc6e>", line 8, in insert_debank_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Library/Frameworks/Pytho

['', '', 'DuckDao']
['', '', 'dYdX']
['', '', 'Emoon']
['', '', 'Emporium']
['', '', 'Equalizer']
['', '', 'Empty Set Gold']
['', '', 'FairSwap']
['', '', 'Fei']
['', '', 'Ferrum']
['', '', 'FIREDAO']
['', '', 'Flexa']
['', '', 'ForTube Bond']
['', '', 'ForTube Bank V1']
['', '', 'GrowthDeFi']
['', '', 'HAKKA']
['', '', 'Harvest']
['', '', 'Tax Heaven']
['', '', 'HubrisOne X']
['', '', 'HyperBC Farm']
['', '', 'ichi.farm']
['', '', 'Yearn']
['', '', 'Incognito']
['', '', 'Index Coop']
['', '', 'Indexed']
['', '', 'InstaDapp']
['', '', 'Inverse']
['', '', 'Keep']
['', '', 'Keep3r']
['', '', 'KIMCHI']
['', '', 'Kwenta']
['', '', 'Kyber']
['', '', 'Leverj']
['', '', 'LIDO']
['', '', 'Loopring V1']
['', '', 'LuaSwap']
['', '', 'Benchmark(MARK)']
['', '', 'Matcha']
['', '', 'Melon']
['', '', 'Mercurity']
['', '', 'Mesa']
['', '', 'MetaMask Swaps']
['', '', 'M.finance']
['', '', 'Mirror']
['', '', 'MITH.CASH']
['', '', 'Mona']
['', '', 'Mooniswap']
['', '', 'mStable']
['', '', 'Mushrooms']
[

In [25]:
db_project_symbol_name_df.shape

(288, 2)

In [26]:
api_key = "3a2957dbae161d9aac23aadb136880f304a1653c2003ca11cc478e2f30f8"
payload_projects = {'api-key': api_key} 
url_projects = 'https://data-api.defipulse.com/api/v1/defipulse/api/GetProjects'
res_proj = requests.get(url_projects,params= payload_projects).json()
defi_plus_project_name = []
for d in res_proj:
    defi_plus_project_name.append([d['name'], d['id']])

In [27]:
defi_plus_project_name_df = pd.DataFrame(defi_plus_project_name, columns=['name','defiplus_proj_id'])
defi_plus_project_name_df['name_in_defi'] = defi_plus_project_name_df['name']


In [28]:
defi_plus_project_name_df

,name,defiplus_proj_id,name_in_defi
0,Maker,0,Maker
1,Compound,1,Compound
2,Aave,22,Aave
3,SushiSwap,43,SushiSwap
4,Uniswap,2,Uniswap
...,...,...,...
78,PEAKDEFI,79,PEAKDEFI
79,DeFiner,50,DeFiner
80,Auctus,38,Auctus
81,UniTrade,70,UniTrade


In [29]:
defip_in_cmc_sn_df = df1[df1['name'].isin(defi_plus_project_name_df['name'])].copy()
defip_in_cmc_sn_df.reset_index(inplace=True)
del defip_in_cmc_sn_df['index']

In [30]:
defip_in_cmc_sn_df

,symbol,name,name_in_cmc
0,UNI,Uniswap,Uniswap
1,AAVE,Aave,Aave
2,SUSHI,SushiSwap,SushiSwap
3,SNX,Synthetix,Synthetix
4,MKR,Maker,Maker
5,COMP,Compound,Compound
6,MATIC,Polygon,Polygon
7,YFI,yearn.finance,yearn.finance
8,BNT,Bancor,Bancor
9,LRC,Loopring,Loopring


In [31]:
def insert_defi_coin_cmc_symbol(df):
    connection = __connect__('defiplus')
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_defi']])
            sql = "INSERT INTO `defi_symbols` (`symbol`, `name_in_cmc`, `name_in_defi`) VALUES (%s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_defi']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

In [32]:
defi_cmc_sybols_df = pd.merge(defip_in_cmc_sn_df, defi_plus_project_name_df, on='name', how='outer', suffixes=('_in_cmc', '_in_debank'), indicator=True)

defi_cmc_sybols_df = defi_cmc_sybols_df.astype(object).where(pd.notnull(defi_cmc_sybols_df), "")
insert_defi_coin_cmc_symbol(defi_cmc_sybols_df)

['UNI', 'Uniswap', 'Uniswap']
['AAVE', 'Aave', 'Aave']
['SUSHI', 'SushiSwap', 'SushiSwap']
['SNX', 'Synthetix', 'Synthetix']
['MKR', 'Maker', 'Maker']
['COMP', 'Compound', 'Compound']
['MATIC', 'Polygon', 'Polygon']
['YFI', 'yearn.finance', 'yearn.finance']
['BNT', 'Bancor', 'Bancor']
['LRC', 'Loopring', 'Loopring']
['DODO', 'DODO', 'DODO']
['BAL', 'Balancer', 'Balancer']
['BADGER', 'Badger DAO', 'Badger DAO']
['REP', 'Augur', 'Augur']
['GNO', 'Gnosis', 'Gnosis']
['STAKE', 'xDai', 'xDai']
['FARM', 'Harvest Finance', 'Harvest Finance']
['VSP', 'Vesper', 'Vesper']
['TRU', 'TrueFi', 'TrueFi']
['TORN', 'Tornado Cash', 'Tornado Cash']
['IDEX', 'IDEX', 'IDEX']
['DF', 'dForce', 'dForce']
['MPH', '88mph', '88mph']
['PEAK', 'PEAKDEFI', 'PEAKDEFI']
['MET', 'Metronome', 'Metronome']
['FOR', 'ForTube', 'ForTube']
['PICKLE', 'Pickle Finance', 'Pickle Finance']
['AUC', 'Auctus', 'Auctus']
['FIN', 'DeFiner', 'DeFiner']
['SI', 'Siren', 'Siren']
['MCB', 'MCDEX', 'MCDEX']
['YAX', 'yAxis', 'yAxis']
['', 

In [33]:
df = pd.read_csv('Name_Master - Defipulse_missing_from_coinmarketcap.csv')
del df['Unnamed: 0']

In [34]:
df = df[['Coinmarketcap_symbol','Coinmarketcap_name','Defipulse_project']]
df.columns = ['symbol', 'name_in_cmc', 'name_in_defi']

In [35]:
df

,symbol,name_in_cmc,name_in_defi
0,MLN,Enzyme,Enzyme-Finance
1,NaN,NaN,Alpha-Homora
2,AUC,Auctus,Auctus
3,NaN,NaN,B.Protocol
4,BZRX,bZx Protocol,bZx
5,CREAM,Cream Finance,C.R.E.A.M.-Finance
6,CREAM,Cream Finance,C.R.E.A.M.-Swap
7,NaN,NaN,Connext
8,COVER,COVER Protocol,Cover-Protocol
9,CRV,Curve DAO Token,Curve-Finance


In [36]:
df = df.astype(object).where(pd.notnull(df), "")
insert_defi_coin_cmc_symbol(df)

['MLN', 'Enzyme', 'Enzyme-Finance']
['', '', 'Alpha-Homora']
['AUC', 'Auctus', 'Auctus']
Traceback (most recent call last):
  File "<ipython-input-31-52013864bf9f>", line 8, in insert_defi_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in r

Traceback (most recent call last):
  File "<ipython-input-31-52013864bf9f>", line 8, in insert_defi_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Library/Frameworks/Python.

['MCB', 'MCDEX', 'MCDEX']
Traceback (most recent call last):
  File "<ipython-input-31-52013864bf9f>", line 8, in insert_defi_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/

['', '', 'Set-Protocol']
['SI', 'Siren', 'Siren']
Traceback (most recent call last):
  File "<ipython-input-31-52013864bf9f>", line 8, in insert_defi_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection.

In [37]:
df = pd.read_csv('Name_Master - Duneanalytics_missing_from_coinmarketcap.csv')
del df['Unnamed: 0']

In [38]:
df = df[['Coinmarketcap_symbol','Coinmarketcap_name','Duneanalytics']]
df.columns = ['symbol', 'name_in_cmc', 'name_in_duneanalytics']

,symbol,name_in_cmc,name_in_duneanalytics
0,MKR,Maker,MakerDAO
1,NaN,NaN,DeFi
2,NaN,NaN,dYdX
3,KNC,Kyber Network,Kyber
4,NaN,NaN,InstaDApp
5,NaN,NaN,Set Protocol
6,NXS,Nexus,Nexus Mutual
7,NaN,NaN,PoolTogether
8,CRV,Curve DAO Token,Curve
9,CREAM,Cream Finance,Cream


In [71]:
def insert_duneanalytics_coin_cmc_symbol(df):
    connection = __connect__('duneanalytics')
    cursor = get_cursor(connection)
    for i, row in df.iterrows():
        try:
            print([row['symbol'], row['name_in_cmc'], row['name_in_duneanalytics'], row['query']])
            sql = "INSERT INTO `dune_symbols` (`symbol`, `name_in_cmc`, `name_in_duneanalytics`, `query`) VALUES (%s, %s, %s, %s)"
            cursor.execute(sql, (row['symbol'], row['name_in_cmc'], row['name_in_duneanalytics'],  row['query']))
        except Exception:
            print(traceback2.format_exc())
    connection.commit()
    cursor.close()
    connection.close()

In [44]:
df = df.astype(object).where(pd.notnull(df), "")
insert_duneanalytics_coin_cmc_symbol(df)

['MKR', 'Maker', 'MakerDAO']
Traceback (most recent call last):
  File "<ipython-input-40-df46d7a114de>", line 8, in insert_duneanalytics_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet

Traceback (most recent call last):
  File "<ipython-input-40-df46d7a114de>", line 8, in insert_duneanalytics_coin_cmc_symbol
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 148, in execute
    result = self._query(query)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/cursors.py", line 310, in _query
    conn.query(q)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 548, in query
    self._affected_rows = self._read_query_result(unbuffered=unbuffered)
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 775, in _read_query_result
    result.read()
  File "/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/pymysql/connections.py", line 1156, in read
    first_packet = self.connection._read_packet()
  File "/Library/Framework

In [42]:
df

,symbol,name_in_cmc,name_in_duneanalytics
0,MKR,Maker,MakerDAO
1,,,DeFi
2,,,dYdX
3,KNC,Kyber Network,Kyber
4,,,InstaDApp
5,,,Set Protocol
6,NXS,Nexus,Nexus Mutual
7,,,PoolTogether
8,CRV,Curve DAO Token,Curve
9,CREAM,Cream Finance,Cream


In [53]:
url = 'https://explore.duneanalytics.com/api/dashboards/defi-users-over-time'
headers = {'cookie' : 'remember_token=13512-2a54fbb6028321cb22e546f1cfa08d28|9d3068f68295af42a53b383b2927e9aa2ff2c9de17eb2cb70b10dd9833004a0628677fb41bb047d40335bb9c79c87b505800758bf0223e4949b0d631f11b825b; session=.eJwdjsFqg0AURX-lzDoF5zkGEbpIUYcu3hNldHA2QY1WJ3ETKakT8u8durz3ci7nyc7TfdxmlkzdbRsP7LxcWPJkbz1LGMpMkMsEWrMaW3Ny347ALEaXO7mvHdMhIJ3fSJWBUdcQ0xMYba4IPtvPtZDV0urc0y1HWT8KddpJVZZsLdo1t6TrwPdAtrGo0JFqgWT929pmRmkWSrOIVjOjRfD7o0gHYXSz-j_XujJEWTpylXfLPtjrwH628f7vz3gYcXiHLhJT3x8DiEPgQw8wRuI48WHqgvgCMXv9AUJ2UNo.Ex5aOA.vsUA2xDJ7OvU_xmnWToPqPO3jwA'}
data = requests.get(url, headers = headers)
response = data.json()
widgets = response['widgets']
widget = widgets[0]
query_name = []
for widget in widgets:
    query_name.append(widget['visualization']['query']['name'])
    

In [54]:
query_name

['Total 1inch users over time',
 'Total PoolTogether users over time',
 'Total Kyber users over time',
 'Total Hegic users over time',
 'Total KeeperDAO users over time',
 'Total BadgerDAO users over time',
 'Total Ren users over time',
 'Total Curve users over time',
 'Total 0x users over time',
 'Total MakerDAO users over time',
 'Total Set Protocol users over time',
 'Total Aave users over time',
 'Total SushiSwap users over time',
 'Total Cream users over time',
 'Total Bancor users over time',
 'Total dYdX users over time',
 'Total Harvest Finance users over time',
 'Total Balancer users over time',
 'Total yearn.finance users over time',
 'Total Augur users over time',
 'Total InstaDApp users over time',
 'Total DeFi users over time',
 'Total Nexus Mutual users over time',
 'Total Uniswap users over time',
 'Total Compound users over time',
 'Total Synthetix users over time']

In [56]:
symbol_query = []
for query in query_name:
    symbol_query.append([query, query.replace("Total ", '').replace(' users over time', '')])

In [57]:
symbol_query

[['Total 1inch users over time', '1inch'],
 ['Total PoolTogether users over time', 'PoolTogether'],
 ['Total Kyber users over time', 'Kyber'],
 ['Total Hegic users over time', 'Hegic'],
 ['Total KeeperDAO users over time', 'KeeperDAO'],
 ['Total BadgerDAO users over time', 'BadgerDAO'],
 ['Total Ren users over time', 'Ren'],
 ['Total Curve users over time', 'Curve'],
 ['Total 0x users over time', '0x'],
 ['Total MakerDAO users over time', 'MakerDAO'],
 ['Total Set Protocol users over time', 'Set Protocol'],
 ['Total Aave users over time', 'Aave'],
 ['Total SushiSwap users over time', 'SushiSwap'],
 ['Total Cream users over time', 'Cream'],
 ['Total Bancor users over time', 'Bancor'],
 ['Total dYdX users over time', 'dYdX'],
 ['Total Harvest Finance users over time', 'Harvest Finance'],
 ['Total Balancer users over time', 'Balancer'],
 ['Total yearn.finance users over time', 'yearn.finance'],
 ['Total Augur users over time', 'Augur'],
 ['Total InstaDApp users over time', 'InstaDApp'],
 

In [58]:
dune_symbol_query_df = pd.DataFrame(symbol_query, columns=["query","name"])
dune_symbol_query_df['name_in_duneanalytics'] = dune_symbol_query_df['name']

In [63]:
dune_in_cmc_sn_df = df1[df1['name'].isin(dune_symbol_query_df['name'])].copy()
dune_in_cmc_sn_df.reset_index(inplace=True)
del dune_in_cmc_sn_df['index']

In [73]:
dune_cmc_sybols_df = pd.merge(dune_in_cmc_sn_df, dune_symbol_query_df, on='name', how='outer', suffixes=('_in_cmc', '_in_debank'), indicator=True)

dune_cmc_sybols_df = dune_cmc_sybols_df.astype(object).where(pd.notnull(dune_cmc_sybols_df), "")
insert_duneanalytics_coin_cmc_symbol(dune_cmc_sybols_df)


['UNI', 'Uniswap', 'Uniswap', 'Total Uniswap users over time']
['AAVE', 'Aave', 'Aave', 'Total Aave users over time']
['SUSHI', 'SushiSwap', 'SushiSwap', 'Total SushiSwap users over time']
['SNX', 'Synthetix', 'Synthetix', 'Total Synthetix users over time']
['COMP', 'Compound', 'Compound', 'Total Compound users over time']
['YFI', 'yearn.finance', 'yearn.finance', 'Total yearn.finance users over time']
['BNT', 'Bancor', 'Bancor', 'Total Bancor users over time']
['REN', 'Ren', 'Ren', 'Total Ren users over time']
['ZRX', '0x', '0x', 'Total 0x users over time']
['1INCH', '1inch', '1inch', 'Total 1inch users over time']
['BAL', 'Balancer', 'Balancer', 'Total Balancer users over time']
['REP', 'Augur', 'Augur', 'Total Augur users over time']
['HEGIC', 'Hegic', 'Hegic', 'Total Hegic users over time']
['FARM', 'Harvest Finance', 'Harvest Finance', 'Total Harvest Finance users over time']
['ROOK', 'KeeperDAO', 'KeeperDAO', 'Total KeeperDAO users over time']
['CRM', 'Cream', 'Cream', 'Total Cre

In [68]:
dune_cmc_sybols_df

,symbol,name,name_in_cmc,query,name_in_duneanalytics,_merge
0,UNI,Uniswap,Uniswap,Total Uniswap users over time,Uniswap,both
1,AAVE,Aave,Aave,Total Aave users over time,Aave,both
2,SUSHI,SushiSwap,SushiSwap,Total SushiSwap users over time,SushiSwap,both
3,SNX,Synthetix,Synthetix,Total Synthetix users over time,Synthetix,both
4,COMP,Compound,Compound,Total Compound users over time,Compound,both
5,YFI,yearn.finance,yearn.finance,Total yearn.finance users over time,yearn.finance,both
6,BNT,Bancor,Bancor,Total Bancor users over time,Bancor,both
7,REN,Ren,Ren,Total Ren users over time,Ren,both
8,ZRX,0x,0x,Total 0x users over time,0x,both
9,1INCH,1inch,1inch,Total 1inch users over time,1inch,both
